<a href="https://colab.research.google.com/github/VictorHo1114/Medical-Chatbot/blob/main/RobotMedical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
# 安裝最新 Unsloth 依賴
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install opencc-python-reimplemented # 用於簡轉繁

In [ ]:
import os
from google.colab import drive
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from opencc import OpenCC
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 0. 確保 Google Drive 已掛載 (最優先執行)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 1. 設定模型參數 (統一在這裡設定)
max_seq_length = 1024 # ✂️ 關鍵：直接鎖死 1024，確保跑得動
dtype = None
load_in_4bit = True

# 2. 載入 Qwen 2.5 3B (只載入這一次！)
print("🔄 正在載入模型...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. 加上 LoRA 適配器
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# 4. 準備數據與繁體轉換
print("📚 正在處理數據集 (含繁體轉換)...")
cc = OpenCC('s2twp') # 簡轉繁 (台灣正體)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
你是一位專業的醫療AI助手。請詳細進行推理並回答使用者的醫療問題。
**請務必使用繁體中文（Traditional Chinese）回答。**

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for instruction, cot, output in zip(instructions, cots, outputs):
        # 組合推理鏈 (CoT) 與最終結論
        full_response = f"{cot}\n\n結論：{output}"

        # --- ⚡ 即時繁體轉換 ---
        instruction_tc = cc.convert(instruction)
        full_response_tc = cc.convert(full_response)

        text = alpaca_prompt.format(instruction_tc, full_response_tc) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

# 載入數據集 (記得選 "zh")
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

# 5. 設定訓練參數 (極速版)
print("⚙️ 設定訓練參數...")
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length, # 這裡會自動使用上面設定的 1024
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4, # 長度減半後，Batch 4 應該跑得動
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 200,         # 只跑 200 步
        learning_rate = 3e-4,    # 較高的學習率
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,

        # --- 💾 存檔保險 ---
        output_dir = "/content/drive/MyDrive/Medical_Qwen_Checkpoints",
        save_strategy = "steps",
        save_steps = 50,
        save_total_limit = 2,
    ),
)

# 6. 開始訓練
print("🚀 開始極速訓練！")
trainer_stats = trainer.train()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


/usr/local/lib/python3.12/dist-packages/unsloth/models/rl_replacements.py:946: UserWarning: You are importing from 'trl.experimental'. APIs here are unstable and may change or be removed without notice. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  import trl.experimental.openenv.utils as openenv_utils


🔄 正在載入模型...
==((====))==  Unsloth 2025.12.4: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

...

wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.053700
20,1.669900
30,1.601500
40,1.593100
50,1.551800
60,1.546000
70,1.579300
80,1.545400
90,1.547800
100,1.529600


wandb: WARNING URL not available in offline run
wandb: WARNING URL not available in offline run
wandb: WARNING URL not available in offline run
wandb: WARNING URL not available in offline run


train/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇███
train/grad_norm,█▄▃▂▂▂▂▁▁▂▁▂▂▂▂▂▂▂▁▂
train/learning_rate,███▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▁▁
train/loss,█▃▂▂▂▂▂▂▂▁▁▂▂▁▁▁▂▁▁▁
total_flos,4.662095330535014e+16
train/epoch,0.15864
train/global_step,200
train/grad_norm,0.26874
train/learning_rate,0.0
train/loss,1.5375


In [ ]:
import os

# 設定你剛剛代碼中的存檔路徑 (如果不確定，我們先搜整個 Checkpoints 資料夾)
# 根據你上一段代碼，路徑應該是這個：
checkpoints_path = "/content/drive/MyDrive/Medical_Qwen_Checkpoints"

print(f"📂 正在檢查路徑：{checkpoints_path}")

if os.path.exists(checkpoints_path):
    files = os.listdir(checkpoints_path)
    print(f"✅ 找到了！資料夾內有 {len(files)} 個物件。")
    print("內容包含：", files)

    # 檢查是否有最新的 checkpoint (例如 checkpoint-200)
    checkpoints = [f for f in files if f.startswith("checkpoint-")]
    if checkpoints:
        print(f"🏆 成功存檔的檢查點：{sorted(checkpoints)}")
    else:
        print("⚠️ 有資料夾，但沒看到 checkpoint-XXX，請檢查內容。")
else:
    print("❌ 沒找到路徑！可能是路徑打錯，或還沒同步。")

📂 正在檢查路徑：/content/drive/MyDrive/Medical_Qwen_Checkpoints
✅ 找到了！資料夾內有 4 個物件。
內容包含： ['runs', 'README.md', 'checkpoint-150', 'checkpoint-200']
🏆 成功存檔的檢查點：['checkpoint-150', 'checkpoint-200']


In [1]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace your_notebook.ipynb

[NbConvertApp] WARNING | pattern 'your_notebook.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
 

In [ ]:
# 1. 切換到推理模式
FastLanguageModel.for_inference(model)

# 2. 準備問題 (測試它的醫療知識與繁體能力)
question = "醫生，我最近早上起床都會頭暈，而且手指尖有點麻麻的，這是中風的前兆嗎？我很擔心。"

# 3. 格式化輸入
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
你是一位專業的醫療AI助手。請詳細進行推理並回答使用者的醫療問題。
**請務必使用繁體中文（Traditional Chinese）回答。**

### Input:
{}

### Response:
"""

# 4. 生成回答
inputs = tokenizer(
    [alpaca_prompt.format(question)],
    return_tensors = "pt"
).to("cuda")

print("🤖 你的專屬醫療 AI 正在思考中...\n" + "="*30)

outputs = model.generate(
    **inputs,
    max_new_tokens = 512, # 讓它多講一點
    use_cache = True,
    temperature = 0.7,
)

# 5. 解碼並顯示
result = tokenizer.batch_decode(outputs)[0]
# 只顯示回答的部分
print(result.split("### Response:\n")[-1].replace("<|endoftext|>", ""))

🤖 你的專屬醫療 AI 正在思考中...
哦，早晨起床頭暈，這聽起來好像不太妙。我記得有時候這種情況可能是因為缺血，比如腦血管的問題。手指尖麻麻的，這通常也是個警訊，可能和神經有關。

中風這個詞，我得好好想想。中風可不簡單，它可是個大病。它有兩種型別：一種是腦出血，一種是腦梗塞，還有些叫腔隙性腦梗塞，聽起來還蠻複雜的。

先來看看，頭暈和手指麻木，這兩個症狀確實有些像中風的徵兆。但單憑這兩個，怎麼就斷定一定是中風呢？不能草率下結論，畢竟很多其他狀況也會引起這些症狀。

所以，我覺得有必要去醫院檢查一下，比如做個腦部CT或者MRI，這樣才能搞清楚到底是不是中風。這樣做不僅能排除掉那些假象，還能及早發現問題。

嗯，總之，雖然這些症狀讓人擔心，但不能一概而論就是中風。還是得靠專業的醫學檢查來確認，這樣才能安心。希望我的想法對你有幫助，祝好運！

結論：您描述的症狀，如早晨起床頭暈和手指尖麻麻，確實有可能與中風有關。不過，僅憑這些症狀無法確診為中風，需要進一步的專業檢查來確認。建議您儘快前往醫院進行相關檢查，例如腦部CT或MRI，以排除或確認中風的可能性。此外，保持良好的生活習慣，如戒煙限酒、控制高血壓和血糖等，也有助於預防中風。希望您健康！<|im_end|>
